# James' Version of Scraping.ipynb following Jeff's Workflow

**BOOKMARKS/TABLE OF CONTENTS:**

[Bookmark: 12/23/19](#📓BOOKMARK-FOR-JAMES-&-JEFF)

### To Do:
- Get links from `custom-badges`
    - Use Jeff's code to scrapen the lesson completion from learn
    - Add Portfolio project page extraction (also in custom-badges)

___

In [1]:
!pip install -U fsds_100719
from fsds_100719.imports import *

fsds_1007219  v0.5.13 loaded.  Read the docs: https://fsds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds_100719,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


In [2]:
import requests
import time

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
driver = webdriver.Chrome()

url = 'https://instruction.learn.co/staff/students'

driver.get(url)

## Username for Github

In [4]:
with open("/Users/jamesirving/.secret/learn_login.json") as f:
    import json
    login_info = json.loads(f.read())
login_info.keys()

dict_keys(['username', 'password'])

In [5]:
username = driver.find_element_by_xpath('//*[@id="login_field"]')
username.send_keys(login_info['username'])

## Password for Github

In [6]:
password = driver.find_element_by_xpath('//*[@id="password"]')
password.send_keys(login_info['password'])

In [7]:
sign_in = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
sign_in.click()

In [ ]:
# click view students to get to the students section
view = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div[1]/div/div[2]/a')
view.click()

## [i/o] Can jump down to loading in student dataframe

In [ ]:
stop

### Active students (JMI)

In [ ]:
# grab canceled students
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
active_url = []
active_name = []
for x in soup.find(id = 'js-students-active').find_all('a'):
    print(x['href'] + ' - ' + x.text)
    active_url.append(x['href'])
    active_name.append(x.text)
    

### Compare Active vs Roster

In [ ]:
df_roster = pd.read_csv('cohort_rosters/online-ds-ft_pt-100719 planing - Cohort Roster.csv')
df_roster['full_names']  = df_roster['First Name'] + ' ' +df_roster['Last Name']
df_roster

In [ ]:
df_roster.set_index('full_names',inplace=True)

In [ ]:
try:
    students_to_drop = ['Anila Qureshi','Ameer Saleemi','Jean Robert Malbranche','Ayaan Harper-Shaw']
    df_roster.drop(students_to_drop,inplace=True)
except:
    print('Could not drop stuents.')

df_roster

In [ ]:
##
active_students = pd.DataFrame({'name': active_name,'url':active_url})
active_students

#### Notes:
- Name contains email addresses for the later students. 
- Not sure if its an error or related to actual missing data

In [ ]:
my_students = df_roster.index.to_series()
my_students

In [ ]:
active_students['my_student'] = active_students['name'].isin(my_students.values)
active_students

In [ ]:
df_my_students = active_students.loc[active_students['my_student']==True].copy()
df_my_students

## 💾 SAVING/LOADING LIST OF MY STUDENTS TO CSV

In [8]:
import requests
import time

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
# df_my_students.to_csv('jmi_students_from_active_students.csv')
df_my_students = pd.read_csv('jmi_students_from_active_students.csv',index_col=0)
df_my_students

,name,url,my_student
17,Acusio Bivona,/staff/students/coachcooz,True
170,Andrea Osika,/staff/students/andiosika,True
621,Darius Fuller,/staff/students/dfuller22,True
676,Devin Belden,/staff/students/devinbelden,True
699,Douglas Steen,/staff/students/douglaspsteen,True
924,Ice Asortse,/staff/students/dantarshi,True
975,Jamaal Smith,/staff/students/jdsmithwes,True
1109,John Kline,/staff/students/katahdin28,True
1111,John Lemaster,/staff/students/lemaster1,True
1116,John Ofrecio,/staff/students/jofrecio,True


### Further Processing My Students

In [10]:
df = df_my_students.copy()

In [11]:
def make_instructor_learn_url(partial_url):
    """Prepends the base url to relative url for df.apply"""
    base_url = 'https://instruction.learn.co' + partial_url
    return base_url

df['full_url'] = df['url'].apply(make_instructor_learn_url)
df

,name,url,my_student,full_url
17,Acusio Bivona,/staff/students/coachcooz,True,https://instruction.learn.co/staff/students/co...
170,Andrea Osika,/staff/students/andiosika,True,https://instruction.learn.co/staff/students/an...
621,Darius Fuller,/staff/students/dfuller22,True,https://instruction.learn.co/staff/students/df...
676,Devin Belden,/staff/students/devinbelden,True,https://instruction.learn.co/staff/students/de...
699,Douglas Steen,/staff/students/douglaspsteen,True,https://instruction.learn.co/staff/students/do...
924,Ice Asortse,/staff/students/dantarshi,True,https://instruction.learn.co/staff/students/da...
975,Jamaal Smith,/staff/students/jdsmithwes,True,https://instruction.learn.co/staff/students/jd...
1109,John Kline,/staff/students/katahdin28,True,https://instruction.learn.co/staff/students/ka...
1111,John Lemaster,/staff/students/lemaster1,True,https://instruction.learn.co/staff/students/le...
1116,John Ofrecio,/staff/students/jofrecio,True,https://instruction.learn.co/staff/students/jo...


### Now grab info from each student's learn page

In [ ]:
# my_html = driver.page_source
# soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
# ## Example edge case of not having a maximum num of sessions
# coach_max_link = 'https://instruction.learn.co/staff/students/abbyox'
# driver.get(coach_max_link)

In [ ]:
# test_link = df['full_url'].iloc[1]
# driver.get(test_link)

#### Divergence

In [12]:
def get_student_info_from_full_url(full_url):
    """Get the `uk-card-body` text from student's instruction.learn full url"""
    driver.get(full_url)
    my_html = driver.page_source
    student_soup = BeautifulSoup(my_html, 'html.parser')
    tag = student_soup.find('div',attrs={'class':'uk-card-body'})
    student_info = tag#.text
    print('Function used to return the .text, now it returns the tag itself.')
    return student_info


In [74]:
import re
def get_urls_from_custom_badges(student_info_card):
    """Accepts the student_info card from instruction.learn
    profile.  Retreives the urls from the custom-badges div."""
    links = student_info_card.find_all('a',href=True)
    # learn_url = links[1]['href']


    ## List of Links 
    student_links = dict(
        slack_url = [links[0]['href']],
        learn_url = [links[1]['href']],
        github_url = [links[2]['href']],
        mailto_url = [links[3]['href']],
        projects_url = [links[4]['href']], 
        grad_dash_url = [links[5]['href']])
    
    return student_links


def get_student_dict_from_student_info(student_info_tag):
    """Uses regex to parse info from student's
    instruction.learn.co/staff/students/____ page:
    

    Ex Use:
    full_link = df['full_url'].iloc[i]
    student_info_tag =  get_student_info_from_full_url(full_link)
    
    student_dict = get_student_dict_from_student_info(student_info_tag)
    """
    student_info = student_info_tag.text

    import re

    student_dict = {}
    
    ## Get urls
    student_urls= get_urls_from_custom_badges(student_info_tag)
    for k,v in student_urls.items():
        student_dict[k] = v


    ## Cohort Lead
    re_cohort_lead = re.compile(r"Cohort Lead: (\w* \w*)")
    student_dict['cohort_lead'] = re_cohort_lead.findall(student_info)#[0]

    ## Ed Coach
    re_ed_coach = re.compile(r"Educational Coach: Currently assigned to (\w*.\w*)")
    student_dict['ed_coach'] = re_ed_coach.findall(student_info)#[0]


    ## Last Active
    re_last_active = re.compile(r"last.active.(\d{2,}/\d{2,}/\d{2,})")
    student_dict['last_active'] = re_last_active.findall(student_info)#[0]


    ## Completed Lessons
    re_comp_lessons = re.compile(r"(\d{1,4}).completed lessons")
    student_dict['completed_lessons'] = re_comp_lessons.findall(student_info)#[0]

    ## Joined learn
    re_joined = re.compile(r"Joined Learn (\d{2}/\d{2}/\d{2})")
    student_dict['joined'] = re_joined.findall(student_info)#[0]



    ## Career Coach - MUST BETTER ADDRESS NOT CURRENTLY ASSIGNED
    re_career_coach= re.compile(r"Career Coach: (\w* \w*)")
    student_dict['career_coach'] = re_career_coach.findall(student_info)#[0]

    re_section = re.compile(r"Section: ([A-Za-z\t .]+)")
    student_dict['section'] = re_section.findall(student_info)#[0]
    
    ## Adding Unpacking of lists via dict loop
    
    for k,v in student_dict.items():
        try:
            student_dict[k] = v[0]
        except:
            continue
        
    return student_dict


def get_program_info(program_info_tag):
    
    program_info = program_info_tag.text
    
    program_dict = {}
    
    re_coach_sess= re.compile(r"Touchpoints: (\d{1,2}) (of (\d{1,2}))?") #"Touchpoints: (\d{1,2}) of (\d{1,2})")
    num_ed_coach_sess = re_coach_sess.findall(program_info)
    
    num_used= num_ed_coach_sess[0][0]
    num_avail = num_ed_coach_sess[0][-1]
    
    program_dict['ed_coach_avail'] = num_avail
    program_dict['ed_coach_used'] = num_used

    
    re_cohort = re.compile(r"Cohort: ([A-Za-z-\d]+)")
    program_dict['cohort'] = re_cohort.findall(program_info)[0]
    
    re_pacing = re.compile(r"Pacing: ([A-Za-z-\d ]+)")
    program_dict['pacing'] =re_pacing.findall(program_info)[0]
    
    
    return program_dict
    

def get_profile_info(profile_info_tag):
    
    profile_dict = {}
    profile_text = profile_info_tag.text
    grad_date = re.compile(r"Ideal Graduation Date: (([\d-]?)*)")
    
    profile_dict['grad_date'] = grad_date.findall(profile_text)#[0][0]
    
    
    re_comm_level = re.compile(r"Commitment Level: (\w* Time)")
    profile_dict['commitment_level'] = re_comm_level.findall(profile_text)#[0]
    
    re_lives_in = re.compile(r"Lives in ([A-Za-z ,]+)")
    profile_dict['lives_in'] = re_lives_in.findall(profile_text)#[0]
    return profile_dict

In [14]:
# tags[0].text

In [15]:
"""tags = get_student_instruct_cards(full_url)
student_data_dict =  process_student_instruct_cards(tags)"""

def get_student_instruct_cards(full_url):
    """Get the `uk-card-body` text from student's instruction.learn full url"""
    driver.get(full_url)
    my_html = driver.page_source
    student_soup = BeautifulSoup(my_html, 'html.parser')
    tags = student_soup.find_all('div',attrs={'class':'uk-card-body'})
    return tags

def process_student_instruct_cards(tags_with_cards):
    """Calls on get_student_dict_from_student_info,
    get_program_info, and get_profile_info to make
    student_data_dict.
    """
    student_info_dict = {}
    ### Tags have all 3 boxes
    # blue student_info 
    student_info = tags_with_cards[0]
    student_info_dict['student_info'] = get_student_dict_from_student_info(student_info)

    # red student_info 
    program_info = tags_with_cards[1]
    student_info_dict['program_info'] = get_program_info(program_info)

    # purple profile 
    profile_info = tags_with_cards[2]
    
    student_info_dict['profile_info'] = get_profile_info(profile_info)
    
    return student_info_dict 
    

In [ ]:
# test_link
# tags = get_student_instruct_cards(test_link)

# tags[0].text

In [ ]:
# ## TROUBLESHOOTING
# tags = get_student_instruct_cards(test_link)
# student_info_tag = tags[0]
# student_info = student_info_tag.text

# student_info

In [17]:
def apply_student_info_retrieval(full_url):
    import time 
    time.sleep(2)
    ## Using the streamlined functions
    student_tags = get_student_instruct_cards(full_url)
    student_dict = process_student_instruct_cards(student_tags)


    student_row = pd.Series(student_dict['student_info'])
    for k,v in student_dict['program_info'].items():
        student_row[k]=v

    for k,v in student_dict['profile_info'].items():
        student_row[k]=v    

    return student_row

# BOOKMARK 12/23/19 5:45pm
- Andi doesn't officially have me listed as her cohort lead and caused an error when triyng to get info

- Replaced unpacking of tags from the end of each line to a for loop with at try except clause

In [16]:
import learn_scrape as ls

driver = ls.start_driver()
ls.github_login(driver)

[i] Workflow:
driver = start_driver()
login_data=load_login_data()
github_login(driver,login_data)
instruct_menu_to_cohort_roster(driver,cohort='pt')
df = cohort_driver_to_csv(driver,'pt_cohort_data.csv',load=True)
Loaded json data. Keys:
dict_keys(['username', 'password'])


## WORKING ON GETTING CUSTOM BADGE LINKS

In [18]:
test_link = df['full_url'].iloc[0]
test_link

'https://instruction.learn.co/staff/students/coachcooz'

In [20]:
cards = get_student_instruct_cards(test_link)
cards

[<div class="uk-card-body">
 <div class="custom-badges">
 <a class="custom-badge-icon" href="slack://user?team=learn-co&amp;id=" rel="nofollow" target="_blank">
 <svg height="16" viewbox="0 0 150 150" width="16">
 <path d="M13.533,78.487c-5.494,0.042-10.148-3.232-11.864-8.342c-0.066-0.198-0.123-0.391-0.178-0.583 c-1.87-6.544,1.756-13.391,8.255-15.582l104.24-34.923c1.267-0.363,2.543-0.549,3.807-0.557 c5.64-0.044,10.422,3.303,12.18,8.521l0.156,0.504c1.95,6.815-2.895,12.899-8.7,14.85c-0.004,0.003-1.06,0.36-103.662,35.392 C16.38,78.237,14.957,78.475,13.533,78.487L13.533,78.487z" fill="#70CADB"></path>
 <path d="M30.908,129.752c-5.537,0.04-10.207-3.188-11.903-8.225c-0.064-0.192-0.125-0.385-0.18-0.579 c-1.896-6.622,1.726-13.54,8.247-15.735L131.316,69.97c1.347-0.45,2.738-0.681,4.137-0.693c5.552-0.042,10.43,3.35,12.151,8.445 l0.16,0.529c1.007,3.521,0.412,7.479-1.59,10.602c-1.492,2.321-6.198,4.365-6.198,4.365l-104.64,35.779 c-1.458,0.487-2.945,0.744-4.428,0.757V129.752z" fill="#E01765"></path>


In [21]:
cards[0]

<div class="uk-card-body">
<div class="custom-badges">
<a class="custom-badge-icon" href="slack://user?team=learn-co&amp;id=" rel="nofollow" target="_blank">
<svg height="16" viewbox="0 0 150 150" width="16">
<path d="M13.533,78.487c-5.494,0.042-10.148-3.232-11.864-8.342c-0.066-0.198-0.123-0.391-0.178-0.583 c-1.87-6.544,1.756-13.391,8.255-15.582l104.24-34.923c1.267-0.363,2.543-0.549,3.807-0.557 c5.64-0.044,10.422,3.303,12.18,8.521l0.156,0.504c1.95,6.815-2.895,12.899-8.7,14.85c-0.004,0.003-1.06,0.36-103.662,35.392 C16.38,78.237,14.957,78.475,13.533,78.487L13.533,78.487z" fill="#70CADB"></path>
<path d="M30.908,129.752c-5.537,0.04-10.207-3.188-11.903-8.225c-0.064-0.192-0.125-0.385-0.18-0.579 c-1.896-6.622,1.726-13.54,8.247-15.735L131.316,69.97c1.347-0.45,2.738-0.681,4.137-0.693c5.552-0.042,10.43,3.35,12.151,8.445 l0.16,0.529c1.007,3.521,0.412,7.479-1.59,10.602c-1.492,2.321-6.198,4.365-6.198,4.365l-104.64,35.779 c-1.458,0.487-2.945,0.744-4.428,0.757V129.752z" fill="#E01765"></path>
<path 

In [24]:
test_dict = get_student_dict_from_student_info(cards[0])
test_dict

{'cohort_lead': 'James Irving',
 'ed_coach': 'Dara Paoletti',
 'last_active': '12/23/19',
 'completed_lessons': '207',
 'joined': '10/07/19',
 'career_coach': 'Not currently',
 'section': 'Data Science'}

In [25]:
fs.ihelp(get_student_dict_from_student_info)

------------------------------------------------------------------------------------
------ HELP ------------------------------------------------------------------------

Help on function get_student_dict_from_student_info in module __main__:

get_student_dict_from_student_info(student_info_tag)
    Uses regex to parse info from student's
    instruction.learn.co/staff/students/____ page:
    
    
    Ex Use:
    full_link = df['full_url'].iloc[i]
    student_info_tag =  get_student_info_from_full_url(full_link)
    
    student_dict = get_student_dict_from_student_info(student_info_tag)

------------------------------------------------------------------------------------
------ SOURCE ----------------------------------------------------------------------


```python
def get_student_dict_from_student_info(student_info_tag):
    """Uses regex to parse info from student's
    instruction.learn.co/staff/students/____ page:
    

    Ex Use:
    full_link = df['full_url'].iloc[i]
    student_info_tag =  get_student_info_from_full_url(full_link)
    
    student_dict = get_student_dict_from_student_info(student_info_tag)
    """
    student_info = student_info_tag.text

    import re

    student_dict = {}

    ## Cohort Lead
    re_cohort_lead = re.compile(r"Cohort Lead: (\w* \w*)")
    student_dict['cohort_lead'] = re_cohort_lead.findall(student_info)#[0]

    ## Ed Coach
    re_ed_coach = re.compile(r"Educational Coach: Currently assigned to (\w*.\w*)")
    student_dict['ed_coach'] = re_ed_coach.findall(student_info)#[0]


    ## Last Active
    re_last_active = re.compile(r"last.active.(\d{2,}/\d{2,}/\d{2,})")
    student_dict['last_active'] = re_last_active.findall(student_info)#[0]


    ## Completed Lessons
    re_comp_lessons = re.compile(r"(\d{1,4}).completed lessons")
    student_dict['completed_lessons'] = re_comp_lessons.findall(student_info)#[0]

    ## Joined learn
    re_joined = re.compile(r"Joined Learn (\d{2}/\d{2}/\d{2})")
    student_dict['joined'] = re_joined.findall(student_info)#[0]



    ## Career Coach - MUST BETTER ADDRESS NOT CURRENTLY ASSIGNED
    re_career_coach= re.compile(r"Career Coach: (\w* \w*)")
    student_dict['career_coach'] = re_career_coach.findall(student_info)#[0]

    re_section = re.compile(r"Section: ([A-Za-z\t .]+)")
    student_dict['section'] = re_section.findall(student_info)#[0]
    
    ## Adding Unpacking of lists via dict loop
    
    for k,v in student_dict.items():
        try:
            student_dict[k] = v[0]
        except:
            continue
        
    return student_dict

```

In [28]:
### Tags have all 3 boxes
# blue student_info 
student_info_card = cards[0]#.text

# red student_info 
program_info_card = cards[1]

# purple profile 
profile_info_card = cards[2]

In [29]:
student_info_card.text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nlast active 12/23/19\nSection: Data Science\n\nEducational Coach: Currently assigned to Dara Paoletti\n\nEducational Coach Status Changes?\n\nCareer Coach: Not currently assigned\n Cohort Lead: James Irving\n\n          207 completed lessons (30%, 0 questions asked)\nJoined Learn 10/07/19 (3 months ago)\nStarted FSWD 10/07/19 (3 months ago)\n'

In [38]:
# student_info_card

## Scraping links from 'custom-badges'

In [73]:
test_student = get_student_dict_from_student_info(
    student_info_card)
test_student

{'slack_url': 'slack://user?team=learn-co&id=',
 'learn_url': 'http://learn.co/coachcooz',
 'github_url': 'http://github.com/coachcooz',
 'mailto_url': 'mailto:acusio13@gmail.com',
 'projects_url': 'https://learn.co/portfolio-projects/student-dashboard/coachcooz',
 'grad_dash_url': 'https://learn.co/graduation/student-dashboard/coachcooz',
 'cohort_lead': 'James Irving',
 'ed_coach': 'Dara Paoletti',
 'last_active': '12/23/19',
 'completed_lessons': '207',
 'joined': '10/07/19',
 'career_coach': 'Not currently',
 'section': 'Data Science'}

In [75]:
# if 'urls' in test_student:
#     student_urls = test_student['urls']
#     test_student.pop('urls')

# for k,v in student_urls.items():
#     print(f"{k}: {v}")
#     test_student[k] = v

In [76]:
test_student

{'slack_url': 'slack://user?team=learn-co&id=',
 'learn_url': 'http://learn.co/coachcooz',
 'github_url': 'http://github.com/coachcooz',
 'mailto_url': 'mailto:acusio13@gmail.com',
 'projects_url': 'https://learn.co/portfolio-projects/student-dashboard/coachcooz',
 'grad_dash_url': 'https://learn.co/graduation/student-dashboard/coachcooz',
 'cohort_lead': 'James Irving',
 'ed_coach': 'Dara Paoletti',
 'last_active': '12/23/19',
 'completed_lessons': '207',
 'joined': '10/07/19',
 'career_coach': 'Not currently',
 'section': 'Data Science'}

### Now Make a Function to .apply to df

In [77]:
df.set_index('name',inplace=True)#@['full_url']#inplace=True)
df

,url,my_student,full_url
name,,,
Acusio Bivona,/staff/students/coachcooz,True,https://instruction.learn.co/staff/students/co...
Andrea Osika,/staff/students/andiosika,True,https://instruction.learn.co/staff/students/an...
Darius Fuller,/staff/students/dfuller22,True,https://instruction.learn.co/staff/students/df...
Devin Belden,/staff/students/devinbelden,True,https://instruction.learn.co/staff/students/de...
Douglas Steen,/staff/students/douglaspsteen,True,https://instruction.learn.co/staff/students/do...
Ice Asortse,/staff/students/dantarshi,True,https://instruction.learn.co/staff/students/da...
Jamaal Smith,/staff/students/jdsmithwes,True,https://instruction.learn.co/staff/students/jd...
John Kline,/staff/students/katahdin28,True,https://instruction.learn.co/staff/students/ka...
John Lemaster,/staff/students/lemaster1,True,https://instruction.learn.co/staff/students/le...


# MAKING A LIST OF URLS FOR PROGRESS SCRAPING

In [78]:
STUDENTS = dict(df['full_url'])
STUDENTS

{'Acusio Bivona': 'https://instruction.learn.co/staff/students/coachcooz',
 'Andrea Osika': 'https://instruction.learn.co/staff/students/andiosika',
 'Darius Fuller': 'https://instruction.learn.co/staff/students/dfuller22',
 'Devin Belden': 'https://instruction.learn.co/staff/students/devinbelden',
 'Douglas Steen': 'https://instruction.learn.co/staff/students/douglaspsteen',
 'Ice Asortse': 'https://instruction.learn.co/staff/students/dantarshi',
 'Jamaal Smith': 'https://instruction.learn.co/staff/students/jdsmithwes',
 'John Kline': 'https://instruction.learn.co/staff/students/katahdin28',
 'John Lemaster': 'https://instruction.learn.co/staff/students/lemaster1',
 'John Ofrecio': 'https://instruction.learn.co/staff/students/jofrecio',
 'Melvin Gonzalez': 'https://instruction.learn.co/staff/students/melvingonzalezjr',
 'Michael Burak': 'https://instruction.learn.co/staff/students/michaelburak',
 'Ru Kein': 'https://instruction.learn.co/staff/students/hakkeray',
 'Susanna Han': 'https

In [79]:
for k,v in STUDENTS.items():
    STUDENTS[k] = apply_student_info_retrieval(v)
STUDENTS

{'Acusio Bivona': slack_url                               slack://user?team=learn-co&id=
 learn_url                                    http://learn.co/coachcooz
 github_url                                 http://github.com/coachcooz
 mailto_url                                   mailto:acusio13@gmail.com
 projects_url         https://learn.co/portfolio-projects/student-da...
 grad_dash_url        https://learn.co/graduation/student-dashboard/...
 cohort_lead                                               James Irving
 ed_coach                                                 Dara Paoletti
 last_active                                                   12/23/19
 completed_lessons                                                  207
 joined                                                        10/07/19
 career_coach                                             Not currently
 section                                                   Data Science
 ed_coach_avail                                

# BOOKMARK 12/24/19 12:00pm

In [81]:
type(STUDENTS['Acusio Bivona'])

pandas.core.series.Series

In [85]:
k = 'Acusio Bivona'
data = STUDENTS[k]
df_test = pd.DataFrame(data,columns=[k])
df_test

,Acusio Bivona
slack_url,slack://user?team=learn-co&id=
learn_url,http://learn.co/coachcooz
github_url,http://github.com/coachcooz
mailto_url,mailto:acusio13@gmail.com
projects_url,https://learn.co/portfolio-projects/student-da...
grad_dash_url,https://learn.co/graduation/student-dashboard/...
cohort_lead,James Irving
ed_coach,Dara Paoletti
last_active,12/23/19
completed_lessons,207


In [86]:
for k,v in STUDENTS.items():
    df_test[k] = v
df_test

,Acusio Bivona,Andrea Osika,Darius Fuller,Devin Belden,Douglas Steen,Ice Asortse,Jamaal Smith,John Kline,John Lemaster,John Ofrecio,Melvin Gonzalez,Michael Burak,Ru Kein,Susanna Han,Terry Ollila
slack_url,slack://user?team=learn-co&id=,slack://user?team=learn-co&id=andio (Andi Osika),slack://user?team=learn-co&id=Darius F.,slack://user?team=learn-co&id=Devin B,slack://user?team=learn-co&id=Doug Steen,slack://user?team=learn-co&id=,slack://user?team=learn-co&id=Jamaal Smith,slack://user?team=learn-co&id=John Kline,slack://user?team=learn-co&id=John Lemaster,slack://user?team=learn-co&id=John Ofrecio,"slack://user?team=learn-co&id=Melvin Gonzalez,...",slack://user?team=learn-co&id=michaelburak(the...,slack://user?team=learn-co&id=ru,slack://user?team=learn-co&id=Susanna Han,slack://user?team=learn-co&id=Terry Ollila
learn_url,http://learn.co/coachcooz,http://learn.co/andiosika,http://learn.co/dfuller22,http://learn.co/devinbelden,http://learn.co/douglaspsteen,http://learn.co/dantarshi,http://learn.co/jdsmithwes,http://learn.co/katahdin28,http://learn.co/lemaster1,http://learn.co/jofrecio,http://learn.co/melvingonzalezjr,http://learn.co/michaelburak,http://learn.co/hakkeray,http://learn.co/ohsusanna93,http://learn.co/terryollila
github_url,http://github.com/coachcooz,http://github.com/andiosika,http://github.com/dfuller22,http://github.com/devinbelden,http://github.com/douglaspsteen,http://github.com/dantarshi,http://github.com/jdsmithwes,http://github.com/katahdin28,http://github.com/lemaster1,http://github.com/jofrecio,http://github.com/melvingonzalezjr,http://github.com/michaelburak,http://github.com/hakkeray,http://github.com/ohsusanna93,http://github.com/terryollila
mailto_url,mailto:acusio13@gmail.com,mailto:andiosika@gmail.com,mailto:d_full22@yahoo.com,mailto:devin.belden.28@gmail.com,mailto:douglaspsteen@gmail.com,mailto:dantarshi@gmail.com,mailto:jdsmith1906@gmail.com,mailto:johnmkline@gmail.com,mailto:heylemaster@gmail.com,mailto:john.ofrecio@gmail.com,mailto:mgonzalez713@gmail.com,mailto:michaelrburak@gmail.com,mailto:rukeine@gmail.com,mailto:susanna.jihae.han@gmail.com,mailto:terryollila@gmail.com
projects_url,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...,https://learn.co/portfolio-projects/student-da...
grad_dash_url,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...,https://learn.co/graduation/student-dashboard/...
cohort_lead,James Irving,[],James Irving,James Irving,James Irving,James Irving,James Irving,James Irving,James Irving,James Irving,James Irving,[],James Irving,[],James Irving
ed_coach,Dara Paoletti,Dara Paoletti,Dara Paoletti,Dara Paoletti,Dara Paoletti,Dara Paoletti,Dara Paoletti,Dara Paoletti,Dara Paoletti,Dara Paoletti,Dar

In [89]:
df_students = df_test.T.copy()
df_students.reset_index(inplace=True)
df_students

,index,slack_url,learn_url,github_url,mailto_url,projects_url,grad_dash_url,cohort_lead,ed_coach,last_active,...,joined,career_coach,section,ed_coach_avail,ed_coach_used,cohort,pacing,grad_date,commitment_level,lives_in
0,Acusio Bivona,slack://user?team=learn-co&id=,http://learn.co/coachcooz,http://github.com/coachcooz,mailto:acusio13@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/23/19,...,10/07/19,Not currently,Data Science,11,3,online-ds-pt-100719,Part Time,"[(2020-08-15, )]",[Part Time],"[Los Angeles, California, United States]"
1,Andrea Osika,slack://user?team=learn-co&id=andio (Andi Osika),http://learn.co/andiosika,http://github.com/andiosika,mailto:andiosika@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,[],Dara Paoletti,12/23/19,...,10/07/19,Not currently,Data Science,11,2,online-ds-pt-100719,Part Time,[],[],[]
2,Darius Fuller,slack://user?team=learn-co&id=Darius F.,http://learn.co/dfuller22,http://github.com/dfuller22,mailto:d_full22@yahoo.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/24/19,...,10/07/19,Not currently,Data Science,11,2,online-ds-pt-100719,Part Time,"[(2020-08-31, )]",[Part Time],"[Seattle, Washington, United States]"
3,Devin Belden,slack://user?team=learn-co&id=Devin B,http://learn.co/devinbelden,http://github.com/devinbelden,mailto:devin.belden.28@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/20/19,...,10/07/19,Not currently,Data Science,11,5,online-ds-ft-100719,Full Time,"[(2020-03-31, )]",[Full Time],[United States]
4,Douglas Steen,slack://user?team=learn-co&id=Doug Steen,http://learn.co/douglaspsteen,http://github.com/douglaspsteen,mailto:douglaspsteen@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/21/19,...,10/07/19,Not currently,Data Science,11,3,online-ds-ft-100719,Full Time,"[(2020-03-31, )]",[Full Time],"[Grapevine, Texas, United States]"
5,Ice Asortse,slack://user?team=learn-co&id=,http://learn.co/dantarshi,http://github.com/dantarshi,mailto:dantarshi@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/24/19,...,10/07/19,Not currently,Data Science,11,2,online-ds-ft-100719,Full Time,"[(2020-03-31, )]",[Full Time],"[Pittsburgh, Pennsylvania, United States]"
6,Jamaal Smith,slack://user?team=learn-co&id=Jamaal Smith,http://learn.co/jdsmithwes,http://github.com/jdsmithwes,mailto:jdsmith1906@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/21/19,...,10/07/19,Not currently,Data Science,11,2,online-ds-ft-100719,Full Time,[],[],[]
7,John Kline,slack://user?team=learn-co&id=John Kline,http://learn.co/katahdin28,http://github.com/katahdin28,mailto:johnmkline@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/19/19,...,10/07/19,Not currently,Data Science,11,1,online-ds-ft-100719,Full Time,"[(2020-03-15, )]",[Full Time],[United States]
8,John Lemaster,slack://user?team=learn-co&id=John Lemaster,http://learn.co/lemaster1,http://github.com/lemaster1,mailto:heylemaster@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/24/19,...,10/10/19,Not currently,Data Science,11,2,online-ds-ft-100719,Full Time,"[(2020-03-31, )]",[Full Time],"[Chicago, Illinois, United States]"
9,John Ofrecio,slack://user?team=learn-co&id=John Ofrecio,http://learn.co/jofrecio,http://github.com/jofrecio,mailto:john.ofrecio@gmail.com,https://learn.co/portfolio-projec

### i/o LOADING FULL DF_STUDENTS INFO

In [93]:
# df_students.to_csv('df_students_full_info.csv')
df_students=pd.read_csv('df_students_full_info.csv',
                       index_col=0)
df_students

,index,slack_url,learn_url,github_url,mailto_url,projects_url,grad_dash_url,cohort_lead,ed_coach,last_active,...,joined,career_coach,section,ed_coach_avail,ed_coach_used,cohort,pacing,grad_date,commitment_level,lives_in
0,Acusio Bivona,slack://user?team=learn-co&id=,http://learn.co/coachcooz,http://github.com/coachcooz,mailto:acusio13@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/23/19,...,10/07/19,Not currently,Data Science,11,3,online-ds-pt-100719,Part Time,"[('2020-08-15', '')]",['Part Time'],"['Los Angeles, California, United States']"
1,Andrea Osika,slack://user?team=learn-co&id=andio (Andi Osika),http://learn.co/andiosika,http://github.com/andiosika,mailto:andiosika@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,[],Dara Paoletti,12/23/19,...,10/07/19,Not currently,Data Science,11,2,online-ds-pt-100719,Part Time,[],[],[]
2,Darius Fuller,slack://user?team=learn-co&id=Darius F.,http://learn.co/dfuller22,http://github.com/dfuller22,mailto:d_full22@yahoo.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/24/19,...,10/07/19,Not currently,Data Science,11,2,online-ds-pt-100719,Part Time,"[('2020-08-31', '')]",['Part Time'],"['Seattle, Washington, United States']"
3,Devin Belden,slack://user?team=learn-co&id=Devin B,http://learn.co/devinbelden,http://github.com/devinbelden,mailto:devin.belden.28@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/20/19,...,10/07/19,Not currently,Data Science,11,5,online-ds-ft-100719,Full Time,"[('2020-03-31', '')]",['Full Time'],['United States']
4,Douglas Steen,slack://user?team=learn-co&id=Doug Steen,http://learn.co/douglaspsteen,http://github.com/douglaspsteen,mailto:douglaspsteen@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/21/19,...,10/07/19,Not currently,Data Science,11,3,online-ds-ft-100719,Full Time,"[('2020-03-31', '')]",['Full Time'],"['Grapevine, Texas, United States']"
5,Ice Asortse,slack://user?team=learn-co&id=,http://learn.co/dantarshi,http://github.com/dantarshi,mailto:dantarshi@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/24/19,...,10/07/19,Not currently,Data Science,11,2,online-ds-ft-100719,Full Time,"[('2020-03-31', '')]",['Full Time'],"['Pittsburgh, Pennsylvania, United States']"
6,Jamaal Smith,slack://user?team=learn-co&id=Jamaal Smith,http://learn.co/jdsmithwes,http://github.com/jdsmithwes,mailto:jdsmith1906@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/21/19,...,10/07/19,Not currently,Data Science,11,2,online-ds-ft-100719,Full Time,[],[],[]
7,John Kline,slack://user?team=learn-co&id=John Kline,http://learn.co/katahdin28,http://github.com/katahdin28,mailto:johnmkline@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/19/19,...,10/07/19,Not currently,Data Science,11,1,online-ds-ft-100719,Full Time,"[('2020-03-15', '')]",['Full Time'],['United States']
8,John Lemaster,slack://user?team=learn-co&id=John Lemaster,http://learn.co/lemaster1,http://github.com/lemaster1,mailto:heylemaster@gmail.com,https://learn.co/portfolio-projects/student-da...,https://learn.co/graduation/student-dashboard/...,James Irving,Dara Paoletti,12/24/19,...,10/10/19,Not currently,Data Science,11,2,online-ds-ft-100719,Full Time,"[('2020-03-31', '')]",['Full Time'],"['Chicago, Illinois, United States']"
9,John Ofrecio,slack://user?team=learn-co&id=John Ofrecio,http://learn.co/jofrecio,http://github.com/jofrecio,mailto:

# 📓BOOKMARK: JAMES 

- [ ] Scraping progress from learn.co profiles. (12/24/19 12:30pm)

In [103]:
fs.ihelp(ls.github_login)

------------------------------------------------------------------------------------
------ HELP ------------------------------------------------------------------------

Help on function github_login in module learn_scrape:

github_login(driver, login_data=None)
    Logs into GitHub Account (for instruction.learn)
    url = 'https://instruction.learn.co/staff/students'

------------------------------------------------------------------------------------
------ SOURCE ----------------------------------------------------------------------


```python
def github_login(driver,login_data=None):
    """Logs into GitHub Account (for instruction.learn)
    url = 'https://instruction.learn.co/staff/students'
    """
    if login_data is None:
        login_data= load_login_data()
        
    username = driver.find_element_by_xpath('//*[@id="login_field"]')
    username.send_keys(login_data['username'])

    password = driver.find_element_by_xpath('//*[@id="password"]')
    password.send_keys(login_data['password'])

    sign_in = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
    sign_in.click()

```

### Finding more flexible way of finding login fields

In [106]:
login_soup = BeautifulSoup(driver.page_source)
login_soup.find_all('input')

[<input name="utf8" type="hidden" value="✓"/>,
 <input name="authenticity_token" type="hidden" value="tJrojhvyoXQR8S/lRNFWSDOTBZ1IW/bDBEeeKXBqqRaN3xeSOKr8G1tS4BFSRE7DpW6zBjtn+Pvs/LOXtNvB4A=="/>,
 <input class="input__field" id="user-email" name="user[email]" required="required" type="email"/>,
 <input class="input__field" id="user-password" name="user[password]" required="required" type="password"/>,
 <input class="button button--color-blue button--text-transform-none button--layout-block button--corners-tight" name="commit" type="submit" value="Sign in"/>]

In [111]:
# user_email = driver.find_element_by_id('user-email')
# user_email.click()
# user_email.send_keys(login_info['username'])

In [112]:
# user_password = driver.find_element_by_id('user-password')
# user_password.click()
# user_password.send_keys(login_info['password'])

In [115]:
# submit = driver.find_element_by_xpath('//*[@id="js--region-main"]/div/div[1]/div/div/div[4]/form/div/div[5]/input')
# submit.click()

In [116]:
# import learn_scrape as ls
# login_info = ls.load_login_data()
# ls.github_login(driver,login_info)

In [94]:
from plotly import express as px
import plotly.graph_objects as go

In [117]:
test_url = df_students['learn_url'].iloc[2]
test_url

'http://learn.co/dfuller22'

In [118]:
driver.get(test_url)

# PRIOR SECTIONS:

### Alternative Since can't get program url via Jeff's ` 'uk-card-title uk-margin-remove-bottom'`

In [ ]:
tag = test_soup.find('div',attrs={'class':'uk-card-body'})
student_info = tag.text
student_info


### Using regex to capture fields from student_info


### Example usage of James' functions

In [ ]:
test_url = df['full_url'].iloc[0]
student_info = get_student_info_from_full_url(test_url)
student_dict = get_student_dict_from_student_info(student_info)
student_dict

## 📓BOOKMARK FOR JAMES & JEFF
- Understanding what info Jeff's functions grabs vs info that my functions grab and how to marry the 2

In [ ]:
## JEFF's FUCNTION'
def get_info_full_url(full_url):
#     driver.get('https://instruction.learn.co' + url_part2)
    driver.get(full_url)
    time.sleep(2)
    
    my_html = driver.page_source
    soup = BeautifulSoup(my_html, 'html.parser')
    
    ## Targeting the Red box on instruct.learn profile
    program = soup.findAll('h3', class_ = 'uk-card-title uk-margin-remove-bottom')[1].text.strip()

    ## Searched for text in div's with uk-card-body class
    start_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Educational')
    stop_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Maximum')
    my_str = soup.findAll('div', class_ = 'uk-card-body')[1].text[start_ed_coach:stop_ed_coach]
    ed_coach_taken = [int(s) for s in my_str.split() if s.isdigit()][0]
    try:
        ed_coach_avail = [int(s) for s in my_str.split() if s.isdigit()][1]
    except IndexError:
        ed_coach_avail = ''
    cohort = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[0].text
    pacing = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[1].text
    try:
        past_cohorts = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[2].text
    except IndexError:
        past_cohorts = ''
    learn_link = soup.find('div', class_ = 'custom-badges').findAll('a', class_ = 'custom-badge-icon')[1]['href']
    
    ##James's return
    student_dict = dict( ed_coach_taken=ed_coach_taken,
                        ed_coach_avail = ed_coach_avail,
                        cohort = cohort,
                        pacing=pacing,
                        past_cohorts=past_cohorts,
                        learn_link=learn_link)
    return student_dict
    ### Jeff's Return:
#     return program, ed_coach_taken, ed_coach_avail, cohort, pacing, past_cohorts, learn_link

test_dict =  get_info_full_url(test_link)

In [ ]:
test_dict

### Got up to Scraping Program from student's instruct page

In [ ]:
# test_soup.find('a')

In [ ]:
# res = test_soup.findall('a')

# Jeff's Original Code:

## Cancelled students (JH)

In [ ]:
# # cancelled students
# canceled = driver.find_element_by_xpath('//*[@id="student-index-card"]/h2[2]')
# canceled.click()

In [ ]:
# # grab canceled students
# my_html = driver.page_source
# soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
# canceled_url = []
# canceled_name = []
# for x in soup.find(id = 'js-students-cancelled').find_all('a'):
#     print(x['href'] + ' - ' + x.text)
#     canceled_url.append(x['href'])
#     canceled_name.append(x.text)

In [ ]:
# canceled_df = pd.DataFrame({'name': canceled_name,
#                             'url': canceled_url})

In [ ]:
# canceled_df.head()

## Graduated Students

In [ ]:
# soup.find(id = 'js-students-graduated').find_all('a')

In [ ]:
# graduated_url = []
# graduated_name = []
# for x in soup.find(id = 'js-students-graduated').find_all('a'):
#     print(x['href'] + ' - ' + x.text)
#     graduated_url.append(x['href'])
#     graduated_name.append(x.text)

In [ ]:
# graduated_df = pd.DataFrame({'name': graduated_name,
#                             'url': graduated_url})

In [ ]:
# graduated_df.head()

## Further Processing Cancelled Students

In [ ]:
canceled_df.head()

In [ ]:
canceled_df.loc[0, 'url']

In [ ]:
https://instruction.learn.co/staff/students/siguy12

### Making absolute URLS

In [ ]:
'https://instructor.learn.co' + canceled_df.loc[0, 'url']

In [ ]:
driver.get('https://instruction.learn.co/' + canceled_df.loc[0, 'url'])

### grab program, pacing, past_cohorts, learn.co url

In [ ]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
# program
soup.findAll('h3', class_ = 'uk-card-title uk-margin-remove-bottom')[1].text.strip()

In [ ]:
# ed coach check ins 
soup.findAll('div', class_ = 'uk-card-body')[1]

In [ ]:
soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')

In [ ]:
start_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Educational')

In [ ]:
stop_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Maximum')

In [ ]:
my_str = soup.findAll('div', class_ = 'uk-card-body')[1].text[start_ed_coach:stop_ed_coach]

In [ ]:
my_str

In [ ]:
[int(s) for s in my_str.split() if s.isdigit()]

In [ ]:
# cohort 
soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[0].text

In [ ]:
# pacing
soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[1].text

In [ ]:
# past cohorts 
soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[2].text

In [ ]:
# learn.co link
soup.find('div', class_ = 'custom-badges').findAll('a', class_ = 'custom-badge-icon')[1]['href']

In [ ]:
driver.get('http://learn.co/siguy12')

In [ ]:
def get_info(url_part2):
    driver.get('https://instruction.learn.co' + url_part2)
    time.sleep(2)
    my_html = driver.page_source
    soup = BeautifulSoup(my_html, 'html.parser')
    program = soup.findAll('h3', class_ = 'uk-card-title uk-margin-remove-bottom')[1].text.strip()
    start_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Educational')
    stop_ed_coach = soup.findAll('div', class_ = 'uk-card-body')[1].text.find('Maximum')
    my_str = soup.findAll('div', class_ = 'uk-card-body')[1].text[start_ed_coach:stop_ed_coach]
    ed_coach_taken = [int(s) for s in my_str.split() if s.isdigit()][0]
    try:
        ed_coach_avail = [int(s) for s in my_str.split() if s.isdigit()][1]
    except IndexError:
        ed_coach_avail = ''
    cohort = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[0].text
    pacing = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[1].text
    try:
        past_cohorts = soup.findAll('div', class_ = 'uk-card-body')[1].findAll('div')[2].text
    except IndexError:
        past_cohorts = ''
    learn_link = soup.find('div', class_ = 'custom-badges').findAll('a', class_ = 'custom-badge-icon')[1]['href']
    return program, ed_coach_taken, ed_coach_avail, cohort, pacing, past_cohorts, learn_link

In [ ]:
get_info('/staff/students/siguy12')

In [ ]:
get_info('/staff/students/perafolio')

In [ ]:
canceled_df.head(2)

In [ ]:
prog_list = []
ed_coach_used_list = []
ed_coach_avail_list = []
cohort_list = []
pacing_list = []
past_cohorts_list = []
learn_list = []
for index, x in enumerate(canceled_df['url']):
    prog_elem, ed_elem1, ed_elem2, cohort_elem, pacing_elem, past_elem, learn_elem = get_info(x)
    prog_list.append(prog_elem)
    ed_coach_avail_list.append(ed_elem1)
    ed_coach_used_list.append(ed_elem1)
    cohort_list.append(cohort_elem)
    pacing_list.append(pacing_elem)
    past_cohorts_list.append(past_elem)
    learn_list.append(learn_elem)
    if index % 50 == 0:
        print(f'Completed {index} out of {len(canceled_df["url"])}')

In [ ]:
canceled_df['program'] = prog_list
canceled_df['cohort'] = cohort_list
canceled_df['pacing'] = pacing_list
canceled_df['past_cohorts'] = past_cohorts_list
canceled_df['ed_coach_used'] = ed_coach_used_list
canceled_df['ed_coach_available'] = ed_coach_avail_list
canceled_df['learn_url'] = learn_list

In [ ]:
canceled_df.to_csv('canceled_df.csv', index = False)

In [ ]:
prog_list = []
ed_coach_used_list = []
ed_coach_avail_list = []
cohort_list = []
pacing_list = []
past_cohorts_list = []
learn_list = []
for index, x in enumerate(graduated_df['url']):
    prog_elem, ed_elem1, ed_elem2, cohort_elem, pacing_elem, past_elem, learn_elem = get_info(x)
    prog_list.append(prog_elem)
    ed_coach_avail_list.append(ed_elem1)
    ed_coach_used_list.append(ed_elem1)
    cohort_list.append(cohort_elem)
    pacing_list.append(pacing_elem)
    past_cohorts_list.append(past_elem)
    learn_list.append(learn_elem)
    if index % 50 == 0:
        print(f'Completed {index} out of {len(graduated_df["url"])}')

In [ ]:
graduated_df['program'] = prog_list
graduated_df['cohort'] = cohort_list
graduated_df['pacing'] = pacing_list
graduated_df['past_cohorts'] = past_cohorts_list
graduated_df['ed_coach_used'] = ed_coach_used_list
graduated_df['ed_coach_available'] = ed_coach_avail_list
graduated_df['learn_url'] = learn_list

In [ ]:
graduated_df.to_csv('graduated_df.csv', index = False)

In [ ]:
graduated_df.head()

In [ ]:
graduated_df[graduated_df['name'] == 'Jon Tyler']

In [ ]:
graduated_df['program'].value_counts()

In [ ]:
canceled_df['program'].value_counts()

In [ ]:
56 / 129

In [ ]:
131 / 321

In [ ]:
canceled_df[canceled_df['program'].str.contains('Data Science')]['pacing'].value_counts()

In [ ]:
graduated_df[graduated_df['program'].str.contains('Data Science')]['pacing'].value_counts()

In [ ]:
# data for canceled students 
cancled_sp = canceled_df[(canceled_df['program'].str.contains('Data Science')) & 
           (canceled_df['pacing'].str.contains('Self Paced')) & 
           (canceled_df['past_cohorts'] == '')]

In [ ]:
cancled_sp

In [ ]:
driver.close()

In [ ]:
driver = webdriver.Chrome()

In [ ]:
test_url = cancled_sp.loc[18, 'learn_url']

In [ ]:
driver.get(test_url)

In [ ]:
sign_in_gh = driver.find_element_by_xpath('//*[@id="js--region-main"]/div/div[1]/div/div/div[5]/form/div/div[6]/a')

In [ ]:
sign_in_gh.click()

# SCRAPING LEARN PROFILES FOR PROGRESS 

## Username for Github

In [ ]:
un = driver.find_element_by_xpath('//*[@id="login_field"]')
un.send_keys()

## Password for Github

In [ ]:
pw = driver.find_element_by_xpath('//*[@id="password"]')
pw.send_keys()

In [ ]:
sn = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
sn.click()

In [ ]:
time.sleep(4)

In [ ]:
driver.get(test_url)

In [ ]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
soup.find('h4', class_ = 'heading heading--level-6 heading--color-green heading--font-size-larger heading--weight-bolder').text

In [ ]:
while True:
    try:
        lm = driver.find_element_by_xpath("//*[contains(text(), 'Load more')]")
        lm.click()
        time.sleep(2)
    except:
        break

In [ ]:
driver.get('http://learn.co/govenfist')

In [ ]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
# curriculum
soup.find('h4', class_ = 'heading heading--level-6 heading--color-green heading--font-size-larger heading--weight-bolder').text

In [ ]:
#lessons completed
soup.find('span', class_ = 'heading heading--level-1 heading--color-green heading--font-size-largest heading--weight-lighter').text

In [ ]:
# last lab 
soup.find('h4', class_ = 'heading heading--level-1 heading--color-blue heading--weight-lighter').text

In [ ]:
while True:
    try:
        lm = driver.find_element_by_xpath("//*[contains(text(), 'Load more')]")
        lm.click()
        time.sleep(2)
    except:
        break

In [ ]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [ ]:
first_date = soup.findAll('div', 
                          class_ = 'module module--flush-wings util--padding-tl util--padding-bl')[0]

In [ ]:
# date
first_date.find('div', class_ = 'heading heading--level-2 heading--color-grey-light').text

In [ ]:
lessons = first_date.find('ul',
                          class_ = 'list list--spacing-large list--separators-grey-faintest')

In [ ]:
# lessons
for l_lesson in lessons.findAll('div', class_ = 'media-block__content'):
    print(l_lesson.find('a').text)

In [ ]:
# iterate through the entire 
all_labs = soup.findAll('div', 
                          class_ = 'module module--flush-wings util--padding-tl util--padding-bl')

dates_list = []
lessons_list = []

for d in all_labs:
    date = d.find('div', 
                  class_ = 'heading heading--level-2 heading--color-grey-light').text
    lessons = d.find('ul',
                          class_ = 'list list--spacing-large list--separators-grey-faintest')
    for l_lesson in lessons.findAll('div', class_ = 'media-block__content'):
        dates_list.append(date)
        lessons_list.append(l_lesson.find('a').text)
        print(l_lesson.find('a').text)

In [ ]:
len(dates_list)

In [ ]:
len(lessons_list)

In [ ]:
df_lessons = pd.DataFrame({'dates': dates_list,
                          'lesson': lessons_list})

In [ ]:
df_lessons.tail(15)

In [ ]:
cancled_sp.loc[18, 'url'].split('/')[-1]

In [ ]:
cancled_sp.head()

In [ ]:
cancled_sp['curriculum'] = ''
cancled_sp['lessons_completed'] = 0
cancled_sp['last_lesson'] = ''

In [ ]:
cancled_sp

In [ ]:
#driver.close()
driver = webdriver.Chrome()

driver.get('http://learn.co/superguy200')
    
# sign into github 

# click sign up with github button
sign_in_gh = driver.find_element_by_xpath('//*[@id="js--region-main"]/div/div[1]/div/div/div[5]/form/div/div[6]/a')
sign_in_gh.click()

# enter github username 
un = driver.find_element_by_xpath('//*[@id="login_field"]')
un.send_keys()

# enter github password
pw = driver.find_element_by_xpath('//*[@id="password"]')
pw.send_keys()

# click submit
sn = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
sn.click()

for index, row in cancled_sp.iterrows():
    
    # visit url
    driver.get(row['learn_url'])
    
    # let load
    time.sleep(2)
    
    # grab html
    my_html = driver.page_source
    soup = BeautifulSoup(my_html, 'html.parser')
    
    # check to see if they are in bootcamp prep
    curriculum = soup.find('h4', 
              class_ = 'heading heading--level-6 heading--color-green heading--font-size-larger heading--weight-bolder').text
    print(f'{row["name"]} - {curriculum}')
    if curriculum == 'Bootcamp Prep':
        cancled_sp.loc[index, 'curriculum'] = curriculum
        continue
    else:
        # add curriculum
        cancled_sp.loc[index, 'curriculum'] = curriculum
        
        # grab lessons completed 
        les_completed = soup.find('span', 
                                   class_ = 'heading heading--level-1 heading--color-green heading--font-size-largest heading--weight-lighter').text
        cancled_sp.loc[index, 'lessons_completed'] = les_completed
        
        # grab last lesson completed 
        las_completed = soup.find('h4', 
                                  class_ = 'heading heading--level-1 heading--color-blue heading--weight-lighter').text
        cancled_sp.loc[index, 'last_lesson'] = las_completed
        
        # click load more a bunch
        while True:
            try:
                lm = driver.find_element_by_xpath("//*[contains(text(), 'Load more')]")
                lm.click()
                time.sleep(2)
            except:
                break
                
        # grab html
        time.sleep(2)
        my_html = driver.page_source
        soup = BeautifulSoup(my_html, 'html.parser')
        
        all_labs = soup.findAll('div', 
                          class_ = 'module module--flush-wings util--padding-tl util--padding-bl')

        dates_list = []
        lessons_list = []

        for d in all_labs:
            date = d.find('div', 
                          class_ = 'heading heading--level-2 heading--color-grey-light').text
            lessons = d.find('ul',
                                  class_ = 'list list--spacing-large list--separators-grey-faintest')
            for l_lesson in lessons.findAll('div', class_ = 'media-block__content'):
                dates_list.append(date)
                lessons_list.append(l_lesson.find('a').text)
                
        df_ls = pd.DataFrame({'dates': dates_list,
                          'lesson': lessons_list})
        
        df_ls.to_csv(f'./lessons/canceled/{row["url"].split("/")[-1]}.csv')

### grad_sp

In [ ]:
# graduated students 
grad_sp = graduated_df[(graduated_df['program'].str.contains('Data Science')) & 
            (graduated_df['pacing'].str.contains('Self Pace')) & 
            (graduated_df['name'] != 'Emily Pfeifer')]

In [ ]:
grad_sp['curriculum'] = ''
grad_sp['lessons_completed'] = 0
grad_sp['last_lesson'] = ''

In [ ]:
#driver.close()
driver = webdriver.Chrome()

driver.get('http://learn.co/superguy200')
    
# sign into github 

# click sign up with github button
sign_in_gh = driver.find_element_by_xpath('//*[@id="js--region-main"]/div/div[1]/div/div/div[5]/form/div/div[6]/a')
sign_in_gh.click()

# enter github username 
un = driver.find_element_by_xpath('//*[@id="login_field"]')
un.send_keys()

# enter github password
pw = driver.find_element_by_xpath('//*[@id="password"]')
pw.send_keys()

# click submit
sn = driver.find_element_by_xpath('//*[@id="login"]/form/div[2]/input[8]')
sn.click()

for index, row in grad_sp.iterrows():
    
    # visit url
    driver.get(row['learn_url'])
    
    # let load
    time.sleep(2)
    
    # grab html
    my_html = driver.page_source
    soup = BeautifulSoup(my_html, 'html.parser')
    
    # check to see if they are in bootcamp prep
    curriculum = soup.find('h4', 
              class_ = 'heading heading--level-6 heading--color-green heading--font-size-larger heading--weight-bolder').text
    print(f'{row["name"]} - {curriculum}')
    if curriculum == 'Bootcamp Prep':
        grad_sp.loc[index, 'curriculum'] = curriculum
        continue
    else:
        # add curriculum
        grad_sp.loc[index, 'curriculum'] = curriculum
        
        # grab lessons completed 
        les_completed = soup.find('span', 
                                   class_ = 'heading heading--level-1 heading--color-green heading--font-size-largest heading--weight-lighter').text
        grad_sp.loc[index, 'lessons_completed'] = les_completed
        
        # grab last lesson completed 
        las_completed = soup.find('h4', 
                                  class_ = 'heading heading--level-1 heading--color-blue heading--weight-lighter').text
        grad_sp.loc[index, 'last_lesson'] = las_completed
        
        # click load more a bunch
        while True:
            try:
                lm = driver.find_element_by_xpath("//*[contains(text(), 'Load more')]")
                lm.click()
                time.sleep(2)
            except:
                break
                
        # grab html
        time.sleep(2)
        my_html = driver.page_source
        soup = BeautifulSoup(my_html, 'html.parser')
        
        all_labs = soup.findAll('div', 
                          class_ = 'module module--flush-wings util--padding-tl util--padding-bl')

        dates_list = []
        lessons_list = []

        for d in all_labs:
            date = d.find('div', 
                          class_ = 'heading heading--level-2 heading--color-grey-light').text
            lessons = d.find('ul',
                                  class_ = 'list list--spacing-large list--separators-grey-faintest')
            for l_lesson in lessons.findAll('div', class_ = 'media-block__content'):
                dates_list.append(date)
                lessons_list.append(l_lesson.find('a').text)
                
        df_ls = pd.DataFrame({'dates': dates_list,
                          'lesson': lessons_list})
        
        df_ls.to_csv(f'./lessons/grad/{row["url"].split("/")[-1]}.csv')

In [ ]:
cancled_sp.to_csv('canceled_sp.csv', index = False)
grad_sp.to_csv('grad_sp.csv', index = False)

In [ ]:
grad_sp